In [12]:
import sys
from time import time
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.regression import LabeledPoint
import numpy as np
from pyspark.mllib.evaluation import RegressionMetrics
import math 

In [13]:

def SetLogger( sc ):
    logger = sc._jvm.org.apache.log4j
    logger.LogManager.getLogger("org"). setLevel( logger.Level.ERROR )
    logger.LogManager.getLogger("akka").setLevel( logger.Level.ERROR )
    logger.LogManager.getRootLogger().setLevel(logger.Level.ERROR)

def SetPath(sc):
    global Path
    if sc.master[0:5]=="local" :
        Path="file:/home/hduser/pythonwork/PythonProject/"
    else:
        Path="hdfs://hdnn:8020/user/hduser/"
#如果您要在cluster模式執行(hadoop yarn 或Spark Stand alone)，請依照書上說明，先上傳檔案至HDFS目錄

def extract_label(record):
    label=(record[-1])
    return float(label)

def convert_float(x):
    return (0 if x=="?" else float(x))

def extract_features(record,featureEnd):
    featureSeason=[convert_float(field)  for  field in record[2]] 
    features=[convert_float(field)  for  field in record[4: featureEnd-2]]
    return  np.concatenate( (featureSeason, features))

In [14]:


def PrepareData(sc): 
    #----------------------1.匯入並轉換資料-------------
    print("開始匯入資料...")
    rawDataWithHeader = sc.textFile(Path+"data/hour.csv")
    header = rawDataWithHeader.first() 
    rawData = rawDataWithHeader.filter(lambda x:x !=header)    
    lines = rawData.map(lambda x: x.split(","))
    print (lines.first())
    print("共計：" + str(lines.count()) + "筆")
    #----------------------2.建立訓練評估所需資料 RDD[LabeledPoint]-------------
    labelpointRDD = lines.map(lambda r:LabeledPoint(
                                                    extract_label(r), 
                                                    extract_features(r,len(r) - 1)))

    print (labelpointRDD.first())
    #----------------------3.以隨機方式將資料分為3部份並且回傳-------------
    (trainData, validationData, testData) = labelpointRDD.randomSplit([8, 1, 1])
    print("將資料分trainData:" + str(trainData.count()) + 
             "   validationData:" + str(validationData.count()) +
             "   testData:" + str(testData.count()))
    #print (labelpointRDD.first())
    return (trainData, validationData, testData) #回傳資料

def PredictData(sc,model): 
    #----------------------1.匯入並轉換資料-------------
    print("開始匯入資料...")
    rawDataWithHeader = sc.textFile(Path+"data/hour.csv")
    header = rawDataWithHeader.first() 
    rawData = rawDataWithHeader.filter(lambda x:x !=header)    
    lines = rawData.map(lambda x: x.split(","))
    #print (lines.first())
    print("共計：" + str(lines.count()) + "筆")
    #----------------------2.建立訓練評估所需資料 LabeledPoint RDD-------------
    labelpointRDD = lines.map(lambda r: LabeledPoint(
                                                     extract_label(r), 
                                                     extract_features(r,len(r) - 1)))
    #----------------------3.定義字典----------------
    SeasonDict = { 1 : "春",  2 : "夏",  3 :"秋",  4 : "冬"   }
    HoildayDict={  0 : "非假日", 1 : "假日"  }  
    WeekDict = {0:"一",1:"二",2:"三",3:"四",4 :"五",5:"六",6:"日"}
    WorkDayDict={ 1 : "工作日",  0 : "非工作日"  }
    WeatherDict={ 1 : "晴",  2 : "陰",  3 : "小雨", 4 : "大雨" }
    #----------------------4.進行預測並顯示結果--------------
    for lp in labelpointRDD.take(100):
        predict = int(model.predict(lp.features))
        label=lp.label
        features=lp.features
        result = ("正確" if  (label == predict) else "錯誤")
        error = math.fabs(label - predict)
        dataDesc="  特徵: "+SeasonDict[features[0]] +"季,"+\
                            str(features[1]) + "月," +\
                            str(features[2]) +  "時,"+ \
                            HoildayDict[features[3]] +","+\
                            "星期"+WeekDict[features[4]]+","+ \
                            WorkDayDict[features[5]]+","+\
                            WeatherDict[features[6]]+","+\
                            str(features[7] * 41)+ "度,"+\
                            "體感" + str(features[8] * 50) + "度," +\
                            "溼度" + str(features[9] * 100) + ","+\
                            "風速" + str(features[10] * 67) +\
                            " ==> 預測結果:" + str(predict )+\
                            "  , 實際:" + str(label) + result +",  誤差:" + str(error)
        print (dataDesc)
        

    

def evaluateModel(model, validationData):
    score = model.predict(validationData.map(lambda p: p.features))
    scoreAndLabels=score.zip(validationData.map(lambda p: p.label))
    metrics = RegressionMetrics(scoreAndLabels)
    RMSE=metrics.rootMeanSquaredError
    return( RMSE)
 

def trainEvaluateModel(trainData,validationData,
                                           impurityParm, maxDepthParm, maxBinsParm):
    startTime = time()
    model = DecisionTree.trainRegressor(trainData, 
                                  categoricalFeaturesInfo={}, \
                                  impurity=impurityParm, 
                                  maxDepth=maxDepthParm, 
                                  maxBins=maxBinsParm)
    RMSE = evaluateModel(model, validationData)
    duration = time() - startTime
    print    ("訓練評估：使用參數" + " impurityParm= %s"%impurityParm+ "  maxDepthParm= %s"%maxDepthParm+ "  maxBinsParm = %d."%maxBinsParm + "  結果RMSE = %f " % RMSE) 
    return (RMSE,duration, impurityParm, maxDepthParm, maxBinsParm,model)


def evalParameter(trainData, validationData, evaparm,impurityList, maxDepthList, maxBinsList):
    metrics = [trainEvaluateModel(trainData, validationData,  impurity,maxdepth,  maxBins  ) 
                            for impurity in impurityList 
                            for maxdepth in maxDepthList  
                            for maxBins in maxBinsList ]
    if evaparm=="impurity":
        IndexList=impurityList[:]
    elif evaparm=="maxDepth":
        IndexList=maxDepthList[:]
    elif evaparm=="maxBins":
        IndexList=maxBinsList[:]

    df = pd.DataFrame(metrics,index=IndexList,
                      columns=['RMSE', 'duration','impurityParm', 'maxDepthParm', 'maxBinsParm','model'])
    
    showchart(df,evaparm,'RMSE','duration',0,200 )
    
    
def showchart(df,evalparm ,barData,lineData,yMin,yMax):
    ax = df[barData].plot(kind='bar', title =evalparm,figsize=(10,6),legend=True, fontsize=12)
    ax.set_xlabel(evalparm,fontsize=12)
    ax.set_ylim([yMin,yMax])
    ax.set_ylabel(barData,fontsize=12)
    ax2 = ax.twinx()
    ax2.plot(df[[lineData ]].values, linestyle='-', marker='o', linewidth=2.0,color='r')
    plt.show()
    
def evalAllParameter(training_RDD, validation_RDD, impurityList, maxDepthList, maxBinsList):    
    metrics = [trainEvaluateModel(trainData, validationData,  impurity,maxdepth,  maxBins  ) 
                        for impurity in impurityList 
                        for maxdepth in maxDepthList  
                        for maxBins in maxBinsList ]
    Smetrics = sorted(metrics, key=lambda k: k[0])
    bestParameter=Smetrics[0]
    
    print("調校後最佳參數：impurity:" + str(bestParameter[2]) + 
            "  ,maxDepth:" + str(bestParameter[3]) + 
            "  ,maxBins:" + str(bestParameter[4])   + 
            "  ,結果RMSE = " + str(bestParameter[0]))
    
    return bestParameter[5]

In [15]:
def  parametersEval(training_RDD, validation_RDD):
    
    print("----- 評估maxDepth參數使用 ---------")
    evalParameter(training_RDD, validation_RDD,"maxDepth", 
                              impurityList=["variance"],       
                              maxDepthList =[3, 5, 10, 15, 20, 25]  ,
                              maxBinsList=[10])
    print("----- 評估maxBins參數使用 ---------")
    evalParameter(training_RDD, validation_RDD,"maxBins", 
                              impurityList=["variance"],        
                              maxDepthList=[10],                   
                              maxBinsList=[3, 5, 10, 50, 100, 200 ])  



def CreateSparkContext():
    #sparkConf = SparkConf()                                            \
    #                     .setAppName("RunDecisionTreeRegression")           \
    #                     .set("spark.ui.showConsoleProgress", "false") 
    #sc = SparkContext(conf = sparkConf)
    print ("master="+sc.master)    
    SetLogger(sc)
    SetPath(sc)
    return (sc)

In [16]:
if __name__ == "__main__":
    print("RunDecisionTreeRegression")
    sc=CreateSparkContext()
    print("==========資料準備階段===============")
    (trainData, validationData, testData) =PrepareData(sc)
    trainData.persist(); validationData.persist(); testData.persist()
    print("==========訓練評估階段===============")
    (AUC,duration, impurityParm, maxDepthParm, maxBinsParm,model)= \
             trainEvaluateModel(trainData, validationData, "variance", 10, 100)
    if (len(sys.argv) == 2) and (sys.argv[1]=="-e"):
        parametersEval(trainData, validationData)
    elif   (len(sys.argv) == 2) and (sys.argv[1]=="-a"): 
        print("-----所有參數訓練評估找出最好的參數組合---------")  
        model=evalAllParameter(trainData, validationData,
                          ["variance"],
                          [3, 5, 10, 15, 20, 25], 
                          [3, 5, 10, 50, 100, 200 ])
    print("==========測試階段===============")
    RMSE = evaluateModel(model, testData)
    print("使用testata測試最佳模型,結果 RMSE:" + str(RMSE))
    print("==========預測資料===============")
    PredictData(sc, model)
    #print   model.toDebugString()

RunDecisionTreeRegression
master=spark://spkma:7077
==========資料準備階段===============
開始匯入資料...
['1', '2011-01-01', '1', '0', '1', '0', '0', '6', '0', '1', '0.24', '0.2879', '0.81', '0', '3', '13', '16']
共計：17379筆
(16.0,[1.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0])
將資料分trainData:13938   validationData:1705   testData:1736
==========訓練評估階段===============
訓練評估：使用參數 impurityParm= variance  maxDepthParm= 10  maxBinsParm = 100.  結果RMSE = 81.996031 
==========測試階段===============
使用testata測試最佳模型,結果 RMSE:79.12597035463018
==========預測資料===============
開始匯入資料...
共計：17379筆
  特徵: 春季,1.0月,0.0時,非假日,星期日,非工作日,晴,9.84度,體感14.395度,溼度81.0,風速0.0 ==> 預測結果:40  , 實際:16.0錯誤,  誤差:24.0
  特徵: 春季,1.0月,1.0時,非假日,星期日,非工作日,晴,9.02度,體感13.635度,溼度80.0,風速0.0 ==> 預測結果:40  , 實際:40.0正確,  誤差:0.0
  特徵: 春季,1.0月,2.0時,非假日,星期日,非工作日,晴,9.02度,體感13.635度,溼度80.0,風速0.0 ==> 預測結果:21  , 實際:32.0錯誤,  誤差:11.0
  特徵: 春季,1.0月,3.0時,非假日,星期日,非工作日,晴,9.84度,體感14.395度,溼度75.0,風速0.0 ==> 預測結果:11  , 實際:13.0錯誤,  誤差:2.0
  特徵: 春季,1.0月,4.0時,非假日,星期日,非工作日,晴,9.